# Setting up the question-answering system

In [39]:
from openai import OpenAI
import json
import os
import requests
import argparse

# Establishing connection to OpenAI
openAI_client = OpenAI() # defaults to os.environ.get("OPENAI_API_KEY")
openAI_model = "gpt-4o-mini"

# Establishing connection to you.com search API
you_API_key = os.environ.get("YOU_API_KEY")

def get_ai_web_snippets_for_query(query):
    """ 
    This function uses the you.com search API to get the top 10 search results for a given query.
    The results are returned as a list of dictionaries, where each dictionary contains the title, snippet, and URL of a search result.
    """
    headers = {"X-API-Key": you_API_key}
    params = {"num_web_results":"10","query": query}
    web_snippet_json = requests.get(
        f"https://api.ydc-index.io/search?query={query}",
        params=params,
        headers=headers,
    ).json()
    web_snippet = json.dumps(web_snippet_json)
    
    return web_snippet

def get_response_OpenAI(prompt):
    """
    This function uses the OpenAI API to generate a response to a given prompt.
    The prompt is a list of dictionaries, where each dictionary contains the role and content of a message.
    The response is the content of the message generated by the OpenAI model.
    """
    completion = openAI_client.chat.completions.create(
        model=openAI_model,
        messages=prompt,
        temperature=0,
    )
    return completion.choices[0].message.content

def classify_question(question: str):
    """
    This function uses the OpenAI API to classify a given question into one of 23 medical categories.
    The question is classified based on the content of the question.
    The category is returned as a string.
    """
    system_message = """
    You are a helpful medical assistant. Classify the provided question into one of the 23 categories listed below. Only return the category name.

    23 Categories:

    Association
    Cause
    Complication
    Contraindication
    Diagnosis
    Dosage
    Genetic changes
    Indication
    Information
    Ingredient
    Inheritance
    Interaction
    Organization
    Prevention
    Prognosis
    Resources
    Side effects
    Storage and disposal
    Susceptibility
    Symptom
    Tapering
    Treatment
    Usage
    """
    message_text = [{"role":"system","content":system_message},
                    {"role":"user","content":question}]
    category_of_question = get_response_OpenAI(prompt=message_text)

    return category_of_question

def get_question_focus_to_search_web(question: str):
    system_message = """
    You are a helpful medical assistant. Identify the focus of the question and provide a search query that can be used to find more information in the internet. Only return the search query.

    Example: "What is the treatment for diabetes?" -> "treatment for diabetes"
    """
    message_text = [{"role":"system","content":system_message},
                    {"role":"user","content":question}]
    search_query = get_response_OpenAI(prompt=message_text)

    # Remove the "" from the search query
    search_query = search_query.replace('"', '')

    return search_query

def get_answer_question_with_web_snippets(question: str, web_snippets: str):
    """
    This function generates an answer to a given question using the OpenAI API and the web snippets provided.
    The answer is generated based on the question and the content of the web snippets.
    The answer is returned as a string and contains the relevant URLs from the web snippets.
    """
    base_system_message = """
    You are a helpful medical assistant. Generate an answer to the provided question using the web snippets provided below and your medical knowledge.
    Make sure the answer is relevant to the question and is supported by the web snippets. The web snippets are in JSON format.
    Only return the answer and useful supportive URLs.

    ### WEB SNIPPETS ###
    """
    system_message = base_system_message + "\n\n" + web_snippets
    message_text = [{"role":"system","content":system_message},
                    {"role":"user","content":"Question: \n\n" + question}]
    answer = get_response_OpenAI(prompt=message_text)

    return answer

def answer_question(question: str):
    """
    This function takes a medical question as input and returns an answer to the question.
    The answer is generated using the OpenAI API and the you.com websearch API.
    It also classifies the question into one of 23 medical categories and identifies the focus of the question to search the web.
    """
    # Classify the question into one of the 23 medical categories
    category = classify_question(question)

    # Identify the focus of the question to search the web
    search_query = get_question_focus_to_search_web(question)

    # Get web snippets for the search query
    web_snippets = get_ai_web_snippets_for_query(search_query)

    # Generate an answer using the OpenAI API and the web snippets
    answer = get_answer_question_with_web_snippets(question, web_snippets)
    
    complete_answer = f"Category: {category}\nSearch Query: {search_query}\nAnswer: {answer}"
    
    return category, search_query, web_snippets, answer, complete_answer

# Testing some questions and printing the web snippet afterwards

In [42]:
question = "When and how do you know when you have congenital night blindness?"
category, search_query, web_snippets, answer, complete_answer = answer_question(question)
print(complete_answer)
print("\n\nWEB SNIPPETS: \n\n", web_snippets)

Category: Diagnosis
Search Query: congenital night blindness symptoms and diagnosis
Answer: Congenital night blindness, also known as congenital stationary night blindness (CSNB), is typically identified in childhood, often when a child has difficulty seeing in low light or dark environments. The condition is characterized by several symptoms, including:

1. **Difficulty Seeing in Low Light**: The primary symptom is trouble seeing in dimly lit conditions, which may be noticed by parents or caregivers when the child struggles in dark rooms or during nighttime.

2. **Visual Acuity Issues**: Individuals with CSNB may have reduced visual acuity, which can range from 20/30 to 20/200. This can be assessed through a standard eye examination.

3. **Other Vision Problems**: Additional symptoms may include myopia (nearsightedness), nystagmus (involuntary eye movements), strabismus (crossed eyes), and normal color vision. 

4. **Family History**: Since CSNB is often inherited, a family history of

In [41]:
question = "Is conception a requirement of a molar pregnancy. If so, when ?"
category, search_query, web_snippets, answer, complete_answer = answer_question(question)
print(complete_answer)
print("\n\nWEB SNIPPETS: \n\n", web_snippets)

Category: Diagnosis
Search Query: molar pregnancy conception requirements
Answer: Yes, conception is a requirement for a molar pregnancy. A molar pregnancy occurs when there is a problem during the fertilization process, leading to the development of abnormal tissue instead of a normal embryo and placenta. This genetic error can happen at the time of conception when the sperm fertilizes the egg, resulting in either a complete or partial molar pregnancy. In a complete molar pregnancy, the fertilized egg lacks maternal genetic material, while in a partial molar pregnancy, there may be some fetal tissue, but it is always abnormal and cannot survive.

For more information, you can visit:
- [Cleveland Clinic on Molar Pregnancy](https://my.clevelandclinic.org/health/diseases/17889-molar-pregnancy)
- [NHS on Molar Pregnancy](https://www.nhs.uk/conditions/molar-pregnancy/)
- [American Pregnancy Association on Molar Pregnancy](https://americanpregnancy.org/healthy-pregnancy/birth-defects/molar-

In [43]:
question = "My father is suffering from IBS and is loosing weight day by day.sometimes he even faints due to weakness.He cannot digest food .please help us and suggest us upon this problem."
category, search_query, web_snippets, answer, complete_answer = answer_question(question)
print(complete_answer)
print("\n\nWEB SNIPPETS: \n\n", web_snippets)

Category: Diagnosis
Search Query: IBS weight loss treatment and management
Answer: I'm sorry to hear about your father's condition. It's important to address both his IBS symptoms and the weight loss he is experiencing. Here are some steps you can take to help manage his situation:

1. **Consult a Healthcare Professional**: It's crucial to consult with a doctor or a gastroenterologist who can evaluate his symptoms and provide a tailored treatment plan. Unexplained weight loss and fainting can indicate more serious issues that need medical attention.

2. **Dietary Adjustments**: A low-FODMAP diet may help manage IBS symptoms. This diet involves reducing fermentable carbohydrates that can trigger symptoms. Foods that are generally well-tolerated include bananas, blueberries, oranges, and certain vegetables. However, it's best to work with a nutritionist to ensure he is getting adequate nutrition while following this diet.

3. **Regular Meals**: Encourage him to eat smaller, more frequent